In [1]:
import pandas as pd
from statsmodels.tsa.stattools import grangercausalitytests

# Upload Data in a DataFrame
data = pd.read_csv('./datasets/US data - quarterly.csv', parse_dates=['date'])
data = data[data['date'] >= '1973-04-01']
data.set_index(['date'], inplace=True)
data

,"co2, total","consumption, fossil fuel","consumption, nuclear","consumption, primary","consumption, renewable",gdp chained,"production, fossil fuel","production, nuclear","production, primary","production, renewable"
date,,,,,,,,,,
1973-04-01,1191.325,17.700926,0.201198,19.055534,1.141531,5441.5,14.392755,0.201198,15.735483,1.141531
1973-07-01,1105.705,16.465586,0.211969,17.826532,1.136698,5411.9,14.421367,0.211969,15.770034,1.136698
1973-10-01,1132.106,16.890961,0.252578,18.140034,0.984216,5462.4,14.771822,0.252578,16.008617,0.984216
1974-01-01,1267.861,18.854998,0.261185,20.309980,1.181978,5417.0,14.651743,0.261185,16.094904,1.181978
1974-04-01,1144.458,17.021725,0.268109,18.546133,1.245739,5431.3,14.226263,0.268109,15.740110,1.245739
...,...,...,...,...,...,...,...,...,...,...
2016-04-01,1226.615,18.886686,2.034472,23.643264,2.673412,16663.5,15.932351,2.034472,20.663655,2.696830
2016-07-01,1274.282,19.131206,2.134695,23.952314,2.623535,16778.1,16.251261,2.134695,21.015921,2.629965
2016-10-01,1294.103,19.344535,2.067827,23.909773,2.440118,16851.4,16.642803,2.067827,21.155633,2.445001


In [2]:
from itertools import combinations

maxlag = 20 # theoretical max: 58 -- apply max 20 (5 years)
p_val_max = 0.01

for indicator1, indicator2 in list(combinations(data.columns, r=2)):

    results = []

    for var_caused, var_cause in [(indicator1, indicator2), (indicator2, indicator1)]:
        gct = grangercausalitytests(data[[var_caused, var_cause]], maxlag=maxlag, verbose=False)

        min_p = None
        min_lag = None

        for lag, test in gct.items():
            chi2, p, _, _ = test[0]['params_ftest']

            if min_p == None or p < min_p:
                min_p = p
                min_lag = lag

        results.append({'p':min_p, 'lag':min_lag})

    if results[0]['p'] < p_val_max and results[1]['p'] < p_val_max:
        print(indicator1, "<-->", indicator2, ":", results[0]['lag'], results[1]['lag'])
    elif results[0]['p'] < p_val_max and results[1]['p'] >= p_val_max:
        print(indicator1, "<---", indicator2, ":", results[0]['lag'])
    elif results[0]['p'] >= p_val_max and results[1]['p'] < p_val_max:
        print(indicator1, "--->", indicator2, ":", results[1]['lag'])
    else:
        print(indicator1, "--/--", indicator2)

co2, total <--> consumption, fossil fuel : 3 3
co2, total <--> consumption, nuclear : 3 3
co2, total <--> consumption, primary : 2 2
co2, total <--> consumption, renewable : 3 3
co2, total <--- gdp chained : 4
co2, total <--> production, fossil fuel : 2 6
co2, total <--> production, nuclear : 3 3
co2, total <--> production, primary : 3 2
co2, total <--> production, renewable : 3 3
consumption, fossil fuel <--> consumption, nuclear : 3 3
consumption, fossil fuel <--> consumption, primary : 3 3
consumption, fossil fuel <--> consumption, renewable : 3 3
consumption, fossil fuel <--- gdp chained : 4
consumption, fossil fuel <--> production, fossil fuel : 2 2
consumption, fossil fuel <--> production, nuclear : 3 3
consumption, fossil fuel <--> production, primary : 3 2
consumption, fossil fuel <--> production, renewable : 3 3
consumption, nuclear <--> consumption, primary : 3 3
consumption, nuclear <--> consumption, renewable : 3 3
consumption, nuclear <--> gdp chained : 1 13
consumption, n